In [2]:
!pip uninstall -y numpy
!pip install numpy==1.26

Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6


You can safely remove it manually.
You can safely remove it manually.


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.0-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.0-cp312-cp312-win_amd64.whl (15.5 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cornac 2.3.3 requires numpy>2.0.0, but you have numpy 1.26.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\lucas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install -q google-generativeai


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\lucas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise.model_selection import train_test_split  # ESTA es la correcta
from sklearn.metrics import root_mean_squared_error, mean_squared_error
from collections import defaultdict
import math
from tqdm import tqdm
from surprise import Dataset, Reader, SVD
from surprise import accuracy
import seaborn as sns
import re
import os
import google.generativeai as genai
#import isodate


In [3]:
# Load CSVs
recipes = pd.read_csv("recipes.csv", dtype={'CookTime': str, 'PrepTime': str, 'TotalTime': str, 'RecipeId': int})
reviews = pd.read_csv("reviews.csv")

print(recipes.shape, reviews.shape)
recipes.head()


(522517, 28) (1401982, 8)


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [4]:
print(recipes.shape, reviews.shape)
reviews.head()

(522517, 28) (1401982, 8)


,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z


In [5]:
# Drop duplicate recipes/reviews
recipes = recipes.drop_duplicates(subset="RecipeId")
reviews = reviews.drop_duplicates(subset="ReviewId")

nutritional_cols = [
    "Calories","FatContent","SaturatedFatContent","CholesterolContent",
    "SodiumContent","CarbohydrateContent","FiberContent","SugarContent","ProteinContent"
]
for col in nutritional_cols:
    recipes[col] = pd.to_numeric(recipes[col], errors="coerce")


In [6]:
def get_top_n(predictions, n=10): # Obtenida de https://surprise.readthedocs.io/en/stable/FAQ.html
    """Devuelve el top-N para cada usuario a partir de un conjunto de predicciones."""

    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Ordenar predicciones por score
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [7]:
def precision_recall_at_k(recs, relevant, k=10):
    recs_k = recs[:k]
    hits = len(set(recs_k) & set(relevant))
    precision = hits / k if k else 0
    recall = hits / len(relevant) if relevant else 0
    return precision, recall

def ndcg_at_k(recs, relevant, k=10):
    recs_k = recs[:k]
    dcg = sum(1 / math.log2(i+2) for i, iid in enumerate(recs_k) if iid in relevant)
    idcg = sum(1 / math.log2(i+2) for i in range(min(len(relevant), k)))
    return dcg / idcg if idcg > 0 else 0


In [ ]:
API_KEY = os.getenv("GENAI_API_KEY")
genai.configure(api_key=API_KEY)
for model in genai.list_models():
    print(model.name)


models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
mo

In [ ]:
model = genai.GenerativeModel("gemini-2.0-flash")  

response = model.generate_content("Write a short recipe recommendation for a healthy dinner.")
print(response.text)


## Quick & Easy Lemon Herb Salmon with Roasted Asparagus

**Why it's great:** This recipe is packed with protein, healthy fats, and vitamins. It's also quick to prepare and cooks in under 30 minutes!

**Ingredients:**

*   2 Salmon fillets (about 6 oz each)
*   1 bunch Asparagus, trimmed
*   1 tbsp Olive Oil
*   1 Lemon, halved
*   1 tsp Dried Herbs (Italian blend or Herbs de Provence)
*   Salt and Pepper to taste

**Instructions:**

1.  **Preheat:** Preheat oven to 400°F (200°C).
2.  **Prep Asparagus:** Toss asparagus with 1/2 tbsp olive oil, salt, and pepper. Spread on a baking sheet.
3.  **Prep Salmon:** Place salmon fillets on the same baking sheet. Drizzle with remaining olive oil, sprinkle with herbs, salt, and pepper. Squeeze juice from half a lemon over the salmon.
4.  **Bake:** Bake for 12-15 minutes, or until salmon is cooked through and flakes easily with a fork and asparagus is tender.
5.  **Serve:** Serve immediately with the remaining lemon wedge.

**Optional additions:**

In [9]:
def nutri_score(row):
    # Negative components
    neg = 0
    if not pd.isna(row["Calories"]): neg += min(10, row["Calories"] / 335)  # ~335 kJ = 80 kcal per point
    if not pd.isna(row["SugarContent"]): neg += min(10, row["SugarContent"] / 4.5)
    if not pd.isna(row["SaturatedFatContent"]): neg += min(10, row["SaturatedFatContent"] / 1)
    if not pd.isna(row["SodiumContent"]): neg += min(10, row["SodiumContent"] / 90)

    # Positive components
    pos = 0
    if not pd.isna(row["FiberContent"]): pos += min(5, row["FiberContent"] / 0.9)
    if not pd.isna(row["ProteinContent"]): pos += min(5, row["ProteinContent"] / 1.6)

    score = neg - pos
    if score <= -1: return "A"
    elif score <= 2: return "B"
    elif score <= 10: return "C"
    elif score <= 18: return "D"
    else: return "E"

recipes["NutriScore"] = recipes.apply(nutri_score, axis=1)
print(recipes[["RecipeId", "NutriScore"]].head(20))

    RecipeId NutriScore
0         38          C
1         39          D
2         40          D
3         41          D
4         42          C
5         43          D
6         44          D
7         45          C
8         46          B
9         47          B
10        48          E
11        49          D
12        50          B
13        51          B
14        52          B
15        53          C
16        54          D
17        55          C
18        56          E
19        57          A


In [10]:
def get_top_n(predictions, n=10):
    """Devuelve las N-mejores recomendaciones para cada usuario de un set de predicción.

    Args:
        predictions(lista de objetos Prediction): La lista de predicción obtenida del método test.
        n(int): El número de recomendaciónes por usuario

    Returns:
    Un diccionario donde las llaves son ids de usuario y los valores son listas de tuplas:
        [(item id, rating estimation), ...] de tamaño n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [11]:
def precision_recall_at_k(top_n, test_user_items, k=10):
    precisions = []
    recalls = []
    for uid, recs in top_n.items():
        if uid not in test_user_items:
            continue
        relevant = set(test_user_items[uid])
        recommended = set([iid for iid, _ in recs[:k]])
        precisions.append(len(recommended & relevant) / k)
        recalls.append(len(recommended & relevant) / len(relevant))
    return np.mean(precisions), np.mean(recalls)

def ndcg_at_k(top_n, test_user_items, k=10):
    ndcgs = []
    for uid, recs in top_n.items():
        if uid not in test_user_items:
            continue
        relevant = set(test_user_items[uid])
        dcg = 0
        for i, (iid, _) in enumerate(recs[:k]):
            if iid in relevant:
                dcg += 1 / math.log2(i + 2) 
        idcg = sum(1 / math.log2(i + 2) for i in range(min(k, len(relevant))))
        ndcgs.append(dcg / idcg if idcg > 0 else 0)
    return np.mean(ndcgs)

In [30]:
import re, numpy as np
from tqdm import tqdm
import google.generativeai as genai

class LLMPredictor:
    def __init__(self, recipes, user_histories, batch_size=10):
        self.recipes = recipes
        self.batch_size = batch_size
        self.model = genai.GenerativeModel("gemini-2.0-flash-lite")
        self.user_histories = user_histories
            
    def predict_batch(self, pairs):
        """
        pairs: list of (user_id, recipe_id, true_rating)
        """
        prompt = (
            "You are predicting how much each user would rate a recipe. "
            "Output only a single whole number between 1 and 5 (inclusive) for each user, one per line. "
            "Do not use decimals or fractions. Consider the WHO Health Score into the recommendation, "
            "giving preference to healthier recipes.\n\n"
            "Respond ONLY with one rating per line in the same order.\n\n"
        )

        valid_pairs = []
        for i, (uid, iid, true_r) in enumerate(pairs, 1):
            recipe_row = self.recipes.loc[self.recipes["RecipeId"] == iid]
            if recipe_row.empty:
                continue  # skip unknown recipe IDs

            recipe = recipe_row.iloc[0]
            history = self.user_histories.get(uid, [])
            history_snippet = "\n".join(
                [f"- Recipe: {name}, Rating: {r}" for name, r in history[-5:]]
            ) if history else "No past ratings available."

            prompt += (
                f"{i}. User {uid}:\n"
                f"Past ratings:\n{history_snippet}\n"
                f"Predict rating for this recipe:\n"
                f"Name: {recipe['Name']}, Health score: {recipe['NutriScore']}\n\n"
            )
            valid_pairs.append((uid, iid, true_r))

        if not valid_pairs:
            return []  # nothing to predict

        try:
            response = self.model.generate_content(prompt)
            text = response.text.strip()
            numbers = re.findall(r"\d+", text)
            ratings = [min(max(int(n), 1), 5) for n in numbers]

            # Pad if LLM returned too few
            if len(ratings) < len(valid_pairs):
                ratings += list(np.random.randint(3, 5, len(valid_pairs) - len(ratings)))

            return ratings

        except Exception as e:
            print("Error in batch:", e)
            return list(np.random.randint(3, 5, len(valid_pairs)))


In [26]:
from surprise import accuracy
from sklearn.model_selection import train_test_split



reader = Reader(rating_scale=(1, 5))
data = reviews[["AuthorId", "RecipeId", "Rating"]]
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

recipes["RecipeId"] = recipes["RecipeId"].astype(int)
train_df["RecipeId"] = train_df["RecipeId"].astype(int)
test_df["RecipeId"] = test_df["RecipeId"].astype(int)

# Build user histories
user_histories = {}
grouped = train_df.groupby("AuthorId")
for uid, df in tqdm(grouped):
    # Keep recipe name + rating
    user_histories[uid] = [
        (recipes.loc[recipes["RecipeId"] == rid, "Name"].values[0], int(r))
        for rid, r in zip(df["RecipeId"], df["Rating"])
        if rid in recipes["RecipeId"].values
    ]

100%|██████████| 230674/230674 [04:56<00:00, 777.26it/s] 


In [32]:
recipes["RecipeId"] = recipes["RecipeId"].astype(int)
train_df["RecipeId"] = train_df["RecipeId"].astype(int)
test_df["RecipeId"] = test_df["RecipeId"].astype(int)

# Take a smaller sample of the test set
sample_size = 10000  # adjust depending on patience/budget
sampled_test = test_df.sample(n=sample_size, random_state=42).reset_index(drop=True)

# Initialize model
llm_model = LLMPredictor(recipes, user_histories, batch_size=10)

predictions = []
rows = sampled_test.to_dict(orient="records")

for i in tqdm(range(0, len(rows), llm_model.batch_size)):
    batch = rows[i:i + llm_model.batch_size]
    pairs = [(r["AuthorId"], r["RecipeId"], r["Rating"]) for r in batch]
    ests = llm_model.predict_batch(pairs, )
    for (uid, iid, true_r), est in zip(pairs, ests):
        #print("Estimate for user", uid, "and recipe", iid, "is", est, "true rating is", true_r)
        predictions.append((uid, iid, true_r, est, None))


 25%|██▍       | 246/1000 [02:36<1:09:16,  5.51s/it]

Error in batch: 500 Internal error encountered.


100%|██████████| 1000/1000 [09:52<00:00,  1.69it/s] 


In [34]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def compute_mae(predictions):
    """Compute Mean Absolute Error."""
    true = [t for _, _, t, _, _ in predictions]
    pred = [p for _, _, _, p, _ in predictions]
    return mean_absolute_error(true, pred)

def compute_rmse(predictions):
    """Compute Root Mean Squared Error."""
    true = [t for _, _, t, _, _ in predictions]
    pred = [p for _, _, _, p, _ in predictions]
    return np.sqrt(mean_squared_error(true, pred))

def compute_metrics(predictions):
    mae = compute_mae(predictions)
    rmse = compute_rmse(predictions)
    print(f"MAE: {mae:.3f}")
    print(f"RMSE: {rmse:.3f}")
    return mae, rmse

mae, rmse = compute_metrics(predictions)


MAE: 0.917
RMSE: 1.434


In [36]:
from collections import defaultdict

def get_top_n(predictions, n=10, min_rating=1, max_rating=5):
    """
    Devuelve las N-mejores recomendaciones para cada usuario de un set de predicciones.

    Args:
        predictions (list[tuple]): Lista de tuplas (uid, iid, true_r, est, _)
        n (int): Número de recomendaciones por usuario
        min_rating (float): Valor mínimo de rating permitido
        max_rating (float): Valor máximo de rating permitido

    Returns:
        dict: {uid: [(iid, est), ...]} con las N mejores recomendaciones por usuario
    """
    top_n = defaultdict(list)

    # Asocia todas las predicciones al usuario correspondiente
    for uid, iid, true_r, est, _ in predictions:
        # Ignorar predicciones inválidas o nulas
        if est is None:
            continue
        # Asegurar que el rating esté en rango válido
        est = float(est)
        est = min(max(est, min_rating), max_rating)
        top_n[uid].append((iid, est))

    # Ordenar y recortar las mejores N recomendaciones
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return dict(top_n)


In [37]:
test_user_items = defaultdict(list)
for uid, iid in zip(test_df["AuthorId"], test_df["RecipeId"]):
    test_user_items[uid].append(iid)

In [39]:
top_n = get_top_n(predictions, n=10)
precision, recall = precision_recall_at_k(top_n, test_user_items, k=10)
ndcg = ndcg_at_k(top_n, test_user_items, k=10)

print(f"Precision@10: {precision:.3f}")
print(f"Recall@10:    {recall:.3f}")
print(f"NDCG@10:      {ndcg:.3f}")

Precision@10: 0.149
Recall@10:    0.468
NDCG@10:      0.612
